這份 Notebook 示範 OpenAI Function Calling 的 Extract metadata 用法


In [1]:
from google.colab import userdata
openai_api_key = userdata.get('openai_api_key')

In [2]:
import requests
import json
from pprint import pp

In [3]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=1000, tools=None, tool_choice=None):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }
  if tools:
    payload["tools"] = tools
  if tool_choice:
    payload["tool_choice"] = tool_choice

  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)

  if response.status_code == 200 :
    return obj["choices"][0]["message"] # 改成回傳上一層 message 物件
  else :
    return obj["error"]

## Fake function 用法: 擷取 metadata

透過一個 fake function，目的是拿結構化的 function 參數

In [4]:
content = """大家下午好，歡迎參加台積電2023年第四季財報發表會及電話會議。很高興再次見到大家。我是台積電投資者關係總監 Jeff Su，也是今天的主持人。今天的活動透過台積電網站 www.tsmc.com 進行網路直播，您也可以在其中下載財報發布資料。
今天活動的形式如下：首先，台積電副總裁兼財務長黃文德爾先生將總結我們2023年第四季和2023年全年的營運情況，然後是我們對2024年第一季的指導。黃，台積電首席執行官，魏長城博士；台積電董事長劉馬克博士將共同發表公司的關鍵資訊。然後台積電董事長劉馬克博士將主持問答環節，我們的三位高階主管將回答您的問題。"""


In [5]:
messages = [
  {"role": "system", "content": "You're a metadata extractor." },
  {"role": "user", "content": content }
]

tools = [
    {
        "type": "function",
        "function": {
            "name": "information_extraction",
            "description": "Extracts the relevant information from the passage.",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "公司名稱",
                    },
                    "season": {
                        "type": "string",
                        "description": "報告季度"
                    },
                    "speakers": {
                        "type": "string",
                        "description": "報告人"
                    },
                }
            },
        },
    }
]

# 這個 tool_choice 參數可以要求 GPT 一定要執行某個函數，預設是 auto 讓 GPT 自行判斷
tool_choice =  {"type": "function", "function": {"name": "information_extraction"}}

response = get_completion(messages, tools=tools, tool_choice=tool_choice)
pp(response)

{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_Sm4iu8xQruhwxNcwcdDq6XCN',
                 'type': 'function',
                 'function': {'name': 'information_extraction',
                              'arguments': '{"company_name":"台積電","season":"2023年第四季","speakers":"Jeff '
                                           'Su"}'}}]}


In [6]:
metadata = json.loads(response["tool_calls"][0]["function"]["arguments"])
pp(metadata)

{'company_name': '台積電', 'season': '2023年第四季', 'speakers': 'Jeff Su'}


如果是簡單的 JSON 輸出，不用 function calling 只透過 JSON mode (也許搭配一個 few-shot 範例)，就可以做得很好(可能也比較省 tokens 數)。

不過如果要更嚴格拿到符合 schema 的 JSON 格式，透過 function calling 可以做得更好。
而且，function calling 支援的是 JSON schema 格式: https://json-schema.org/
JSON schema 比你想像中的更厲害...

## 示範使用 Pydantic 來產生 JSON schema

https://docs.pydantic.dev/latest/

Pydantic is the most widely used data validation library for Python.

方便我們將 dict 和 json 做對應轉換


In [7]:
from pydantic import BaseModel, Field
from datetime import date

In [8]:
class Report(BaseModel):
  company_name: str = Field(description="公司名稱")
  season: str = Field(description="報告季度")
  speakers: str = Field(description="報告人")

In [9]:
Report.model_json_schema()

{'properties': {'company_name': {'description': '公司名稱',
   'title': 'Company Name',
   'type': 'string'},
  'season': {'description': '報告季度', 'title': 'Season', 'type': 'string'},
  'speakers': {'description': '報告人', 'title': 'Speakers', 'type': 'string'}},
 'required': ['company_name', 'season', 'speakers'],
 'title': 'Report',
 'type': 'object'}

In [10]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "information_extraction",
            "description": "Extracts the relevant information from the passage.",
            "parameters": Report.model_json_schema()
        },
    }
]

# 這個 tool_choice 參數可以要求 GPT 一定要執行某個函數，預設是 auto 讓 GPT 自行判斷
tool_choice =  {"type": "function", "function": {"name": "information_extraction"}}

response = get_completion(messages, tools=tools, tool_choice=tool_choice)
pp(response)

{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_2tpjOMEa4DpMbx1ryk7sLpht',
                 'type': 'function',
                 'function': {'name': 'information_extraction',
                              'arguments': '{"company_name":"台積電","season":"2023年第四季","speakers":"Jeff '
                                           'Su, 黃文德爾, 魏長城, 劉馬克"}'}}]}


In [11]:
response["tool_calls"][0]["function"]["arguments"]

'{"company_name":"台積電","season":"2023年第四季","speakers":"Jeff Su, 黃文德爾, 魏長城, 劉馬克"}'

In [12]:
report_obj = Report.model_validate_json(response["tool_calls"][0]["function"]["arguments"])

In [13]:
report_obj

Report(company_name='台積電', season='2023年第四季', speakers='Jeff Su, 黃文德爾, 魏長城, 劉馬克')

## 威力加強版

上述只是簡單的 dict，如果是比較複雜的 json 結構呢?

In [14]:
from typing import List

class Speaker(BaseModel):
  name: str

class ReportV2(BaseModel):
  company_name: str = Field(description="公司名稱")
  season: str = Field(description="報告季度")
  speakers: List[Speaker] = Field(description="報告人")

In [15]:
ReportV2.model_json_schema()

{'$defs': {'Speaker': {'properties': {'name': {'title': 'Name',
     'type': 'string'}},
   'required': ['name'],
   'title': 'Speaker',
   'type': 'object'}},
 'properties': {'company_name': {'description': '公司名稱',
   'title': 'Company Name',
   'type': 'string'},
  'season': {'description': '報告季度', 'title': 'Season', 'type': 'string'},
  'speakers': {'description': '報告人',
   'items': {'$ref': '#/$defs/Speaker'},
   'title': 'Speakers',
   'type': 'array'}},
 'required': ['company_name', 'season', 'speakers'],
 'title': 'ReportV2',
 'type': 'object'}

In [16]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "information_extraction",
            "description": "Extracts the relevant information from the passage.",
            "parameters": ReportV2.model_json_schema()
        },
    }
]

tool_choice =  {"type": "function", "function": {"name": "information_extraction"}}

response = get_completion(messages, tools=tools, tool_choice=tool_choice)
pp(response)

{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_OkbDEzR4c3ZWfe8YbfrSusHE',
                 'type': 'function',
                 'function': {'name': 'information_extraction',
                              'arguments': '{"company_name":"台積電","season":"2023年第四季","speakers":[{"name":"Jeff '
                                           'Su"},{"name":"黃文德爾"},{"name":"魏長城博士"},{"name":"劉馬克博士"}]}'}}]}


In [17]:
report_obj = ReportV2.model_validate_json(response["tool_calls"][0]["function"]["arguments"])

In [18]:
report_obj

ReportV2(company_name='台積電', season='2023年第四季', speakers=[Speaker(name='Jeff Su'), Speaker(name='黃文德爾'), Speaker(name='魏長城博士'), Speaker(name='劉馬克博士')])

## Classification 也可以用 Function Calling 做

json schema 支援 enum typ

而且也很方便加上 CoT

回傳是結構化的所以很好拆開

In [111]:
# 這是練習用的 dataset，有 title, description 以及標註的正確分類 category
url = "https://ihower.tw/data/books-dataset-200.csv"
response = requests.get(url)

In [112]:
import pandas as pd
from io import StringIO
pd.read_csv(StringIO(response.text))

,title,description,category,tags
0,統計學：重點觀念與題解 (下) 3/e (適用: 大學用書．高普考．商管所),1. 本書涵蓋範圍既深且廣，難度可輕鬆應付頂尖大學各系所。\r\n2. 本書有各種經濟學與財...,機率統計學 Probability-and-statistics,統計學
1,新時代 乙級數位電子術科含學科題庫實作寶典 - 使用 KiCad/Quartus II / ...,1.依勞動部最新公告試題編而成，專爲有意參加「數位電子乙級技術士技能檢定」之人士編寫，並適合...,技能檢定乙級 Skilltest-b,"技能檢定乙級,考試認證"
2,樂高 SPIKE 機器人創意專題實作 -- 使用 LEGO Education SPIKE ...,1.透過「螺旋式教學」提供一套具有邏輯先後順序教材，由具體到抽象，由簡單到複雜的創客教材。\...,機器人製作 Robots,"樂高,機器人"
3,AI 生成時代：從 ChatGPT 到繪圖、音樂、影片，利用智能創作自我加值、簡化工作，成為...,未來2～5年將是AI 生成的時代\r\n史上用戶最快破億、開啟產業新動能的技術，你跟上了嗎？...,人工智慧,ChatGPT
4,都問 AI 吧！ChatGPT 上手的第一本書,"""寫文案、腳本，編教材，做專案，作業輔助\r\n\r\n都可以問AI，但是——\r\n\r\...",ChatGPT,ChatGPT
...,...,...,...,...
194,Autodesk Civil 3D 2024 from Start to Finish: A...,Master Autodesk Civil 3D 2023 to develop real ...,AutoCAD,AutoDESK
195,Network Programming and Automation Essentials:...,Unleash the power of automation by mastering n...,Go 程式語言,"Go,Python"
196,Internet of Things for Smart Buildings: Levera...,Harness the full potential of IoT in your buil...,物聯網 IoT,物聯網IoT
197,AI and Business Rule Engines for Excel Power U...,A power-packed manual to enhance your decision...,Excel,"Excel,人工智慧"


In [113]:
import csv
import random

f = StringIO(response.text)
reader = csv.DictReader(f)
data = list(reader)

# 隨機選擇20筆資料來做實驗就好了
selected_data = random.sample(data, 20)

# 印出選擇的資料
for row in selected_data:
    print(row)

{'title': 'Rhetoric of InSecurity: The Language of Danger  Fear and Safety in National and International Contexts', 'description': 'This book demands that we question what we are told about security using tools we have had for thousands of years.\r\n\r\nThe work considers the history of security rhetoric in a number of distinct but related contexts including the United States\' security strategy the "war" on Big Tech and current concerns such as cybersecurity. Focusing on the language of security discourse it draws common threads from the ancient world to the present day and the near future. The book grounds recent comparisons of', 'category': '資訊安全', 'tags': '資訊安全'}
{'title': '動手學深度學習 (PyTorch版)(精裝版)', 'description': '本書是《動手學深度學習》的重磅升級版本，選用PyTorch深度學習框架，旨在向讀者交付更為便捷的有關深度學習的交互式學習體驗。\r\n\r\n本書重新修訂《動手學深度學習》的所有內容，並針對技術的發展，新增註意力機制、預訓練等內容。本書包含15章，第一部分介紹深度學習的基礎知識和預備知識，並由線性模型引出基礎的神經網絡——多層感知機；第二部分闡述深度學習計算的關鍵組件、捲積神經網絡、循環神經網絡、註意力機制等大多數現代深度學習應用背後的基本工具；第三部分討論深度學習中常用的優化算法和影響深度學習計算性能的重要因素，並分別列舉深度學習在電

In [114]:
df = pd.DataFrame(selected_data)

In [115]:
category_str = "3D 列印,Arduino,C 程式語言,C++ 程式語言,CI/CD,ChatGPT,Computer Vision,Computer-Science,Computer-architecture,Data Science,Data-visualization,DeepLearning,Domain-Driven Design,Engineer self-growth,Go 程式語言,Information-management,Java,Linux,Machine Learning,Operating-system,Python,R 語言,Reinforcement,UI/UX,Version Control,Visual C#,人工智慧,化學 Chemistry,區塊鏈與金融科技,地理資訊系統 Gis,大數據 Big-data,微軟技術,搜尋引擎優化 SEO,數學,機器人製作 Robots,物聯網 IoT,管理與領導 Management-leadership,網站開發,網頁設計,職涯發展,行動通訊 Mobile-communication,行銷/網路行銷 Marketing,資料庫,資訊安全,軟體工程,軟體架構,軟體測試,雲端運算,電力電子 Power-electronics,電機學 Electric-machinery,電路學 Electric-circuits"
category_arr = category_str.split(",")

In [116]:
from typing import Literal
from typing import Iterable

In [117]:
class Category(BaseModel):
    reasoning: str # CoT
    name: Literal[tuple(category_arr)]

In [118]:
Category.model_json_schema()

{'properties': {'reasoning': {'title': 'Reasoning', 'type': 'string'},
  'name': {'enum': ['3D 列印',
    'Arduino',
    'C 程式語言',
    'C++ 程式語言',
    'CI/CD',
    'ChatGPT',
    'Computer Vision',
    'Computer-Science',
    'Computer-architecture',
    'Data Science',
    'Data-visualization',
    'DeepLearning',
    'Domain-Driven Design',
    'Engineer self-growth',
    'Go 程式語言',
    'Information-management',
    'Java',
    'Linux',
    'Machine Learning',
    'Operating-system',
    'Python',
    'R 語言',
    'Reinforcement',
    'UI/UX',
    'Version Control',
    'Visual C#',
    '人工智慧',
    '化學 Chemistry',
    '區塊鏈與金融科技',
    '地理資訊系統 Gis',
    '大數據 Big-data',
    '微軟技術',
    '搜尋引擎優化 SEO',
    '數學',
    '機器人製作 Robots',
    '物聯網 IoT',
    '管理與領導 Management-leadership',
    '網站開發',
    '網頁設計',
    '職涯發展',
    '行動通訊 Mobile-communication',
    '行銷/網路行銷 Marketing',
    '資料庫',
    '資訊安全',
    '軟體工程',
    '軟體架構',
    '軟體測試',
    '雲端運算',
    '電力電子 Power-electronics',
    '電機學 Electric-ma

In [119]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "classify_category",
            "description": "Classify category",
            "parameters": Category.model_json_schema()
        },
    }
]

tool_choice =  {"type": "function", "function": {"name": "classify_category"}}

Pydantic 還有 Validation 功能

In [120]:
from pydantic import ValidationError

In [121]:
for index, row in df.iterrows():
    messages = [
        {
          "role": "system",
          "content": "請根據以下書籍資訊，選擇最適合的技能分類" },
        {
            "role": "user",
            "content": f"書名: {row['title']} \n 描述: {row['description']}"
        }
    ]
    response = response = get_completion(messages, model="gpt-3.5-turbo", tools=tools, tool_choice=tool_choice)

    try:
      data = Category.model_validate_json(response["tool_calls"][0]["function"]["arguments"])
      df.at[index, 'reasoning'] = data.reasoning
      df.at[index, 'predict'] = data.name
    except ValidationError as e:
      df.at[index, 'predict'] = f"Error: {e}"

df

,title,description,category,tags,reasoning,predict
0,Rhetoric of InSecurity: The Language of Danger...,This book demands that we question what we are...,資訊安全,資訊安全,"This book discusses security rhetoric, fear, a...",R 語言
1,動手學深度學習 (PyTorch版)(精裝版),本書是《動手學深度學習》的重磅升級版本，選用PyTorch深度學習框架，旨在向讀者交付更為便...,DeepLearning,"PyTorch,深度學習",This book focuses on deep learning using the P...,DeepLearning
2,產品經理技術手冊,產品經理工作對人的綜合素養要求很高。比如，硬技能方面的要求有：商業洞察能力、行業分析能力、需...,產品經理,產品經理,產品經理技術手冊涵蓋了產品經理工作所需的硬技能和軟技能，包括商業洞察能力、需求分析能力、產品...,管理與領導 Management-leadership
3,寫給設計師的技術書：從智能終端到感知交互,隨著人機交互和AI技術的快速發展，設計師需要從基於屏幕的圖形用戶界面設計思維，轉變為空間交互...,交互設計 Interaction-design,"交互設計,人工智慧,使用者介面",設計師需要從基於屏幕的圖形用戶界面設計思維轉變為空間交互及智能交互的設計思維，涉及跨設備交互...,UI/UX
4,業務驅動的推薦系統,這是一本從業務視角解讀推薦系統架構設計、評估方法、數據工程和算法原理的著作。\r\n市面上推...,推薦系統,recommender-system,這本書從業務視角出發，探討推薦系統的設計思想、架構、評估方法、數據工程和算法原理，強調將推薦...,Data Science
5,運算放大器權威指南 5/e,運算放大器權威指南（第5版） 出自德州儀器公司前應用工程師之手，書中凝結了作者多年的工作經驗...,Engineer self-growth,"工程師自我提升,電子學,電路學",這本書涵蓋了運算放大器的權威指南，提供了工程師優化模擬電子設計的方法、技術和技巧。書中還包含...,電路學 Electric-circuits
6,Building Modern CLI Applications in Go: Develo...,Evolve the humble CLI using Go and unleash the...,使用者經驗 UX,使用者經驗,This book focuses on developing command-line i...,Go 程式語言
7,Advanced Circuits and Systems for Healthcare a...,VLSI devices downscaling is a very significant...,資訊安全,資訊安全,NaN,Error: 1 validation error for Category\nname\n...
8,深入理解高並發編程：JDK核心技術,本書從實際需求出發，全面細緻地介紹了JDK高並發編程的基礎知識、核心工具和線程池核心技術。\...,Java 程式語言,Java,這本書涵蓋了JDK高並發編程的基礎知識、核心工具和線程池核心技術，讀者可以通過學習提高對高並...,Java
9,NoSQL數據庫技術,本書全面系統地介紹NoSQL數據庫的原理、方法和技術。全書共7章，主要內容包括NoSQL數據...,NoSQL,"NoSQL,SQL",NoSQL數據庫技術的介紹和教學，涵蓋了鍵值數據庫、文檔數據庫、列族數據庫、圖數據庫等相關內容,資料庫


如果你喜歡 Pydantic 這個概念，可以進一步看 jason 針對 openai library 的擴充 https://jxnl.github.io/instructor/
以及他的課程: https://www.wandb.courses/courses/steering-language-models